In [1]:
#libraries
import os
import getpass
import tweepy
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from dotenv import dotenv_values
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from typing import Dict

In [2]:
# env var setup
config = dotenv_values('.env')
API_KEY = config["API_KEY"]
API_SECRET_KEY = config["API_SECRET_KEY"]
ACCESS_TOKEN = config["ACCESS_TOKEN"]
ACCESS_TOKEN_SECRET = config["ACCESS_TOKEN_SECRET"]
OPENAI_API_KEY = config["OPENAI_API_KEY"]
TAVILY_API_KEY = config["TAVILY_API_KEY"]
BEARER_TOKEN = config["BEARER_TOKEN"]

In [3]:
# Authenticate to Twitter
#https://docs.tweepy.org/en/stable/api.html
auth = tweepy.OAuth1UserHandler(API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)
#YnhtM3ZHSDR5UXdmTkpqNzI1M2k6MTpjaQ clientid
#gVKhnQoxdeFLOS6L99Me7UwbFox2rivA7shoj04GRs3dglB9de clientsecret

# Authenticate to OpenAI, setup model
llm = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0, 
    top_p=0.005, 
    api_key=OPENAI_API_KEY
    )

In [4]:
auth = tweepy.Client(
    bearer_token=BEARER_TOKEN, 
    consumer_key=API_KEY, 
    consumer_secret=API_SECRET_KEY, 
    access_token=ACCESS_TOKEN, 
    access_token_secret=ACCESS_TOKEN_SECRET,
    wait_on_rate_limit=True)

In [ ]:
class PostTweetTool:
    def __init__(self, api_key, api_secret_key, access_token, access_token_secret):
        auth = tweepy.OAuth1UserHandler(api_key, api_secret_key, access_token, access_token_secret)
        self.api = tweepy.API(auth)

    def post_tweet(self, message: str) -> str:
        try:
            self.api.update_status(status=message)
            return "Tweet posted successfully!"
        except Exception as e:
            return f"An error occurred: {e}"

In [ ]:
def post_tweet_tool(input_data: Dict) -> Dict:
    message = input_data.get("message", "")
    post_tweet = PostTweetTool(API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    result = post_tweet.post_tweet(message)
    return {"result": result}

In [ ]:
post_tweet = PostTweetTool(API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [ ]:
# tools 
browse_internet = TavilySearchResults(
    max_results=1, 
    )


# Setup agent tools 
tools = [
    browse_internet
    ]

In [ ]:
famous_accounts = """https://x.com/ethermage	Founder of Virtuals
https://x.com/yoheinakajima	BabyAGI founder
https://x.com/_kaitoai	The AI search platform of Web3, bringing you market intelligence in seconds.
https://x.com/luna_virtuals	
https://x.com/virtuals_io	
https://x.com/truth_terminal	The OG crypto A.I. agent, backed by top investors and with top eyes on the project
https://x.com/123skely	Creator of DAOS.fun
https://x.com/daosdotfun	Not A.I. agents, but DAOs for shitcoin investments, including A.I. agents (biggest DAO)
https://x.com/baoskee	DAOS.fun team
https://x.com/brian_armstrong	CEO and founder of Coinbase
https://x.com/Moneytaur_	KOL not A.I. autonomous bots related, but interesting
https://x.com/lmrankhan	Legend investor from Alliance DAO (best accelerator)
https://x.com/alliancedao	Alliance DAO (best accelerator)
https://x.com/QwQiao	Legend investor from Alliance DAO (best accelerator)
https://x.com/pmairca	fake Marc Anderessen account for the biggest A.I. autonomous agent DAO (from DAOS.fun)
https://x.com/Enryu_gfh	
https://x.com/cited	
https://x.com/grimes_v1	
https://x.com/DNS_ERR	
https://x.com/theshikhai/	
https://x.com/Mika_Chasm	Partner at LongHash capital
https://x.com/emmacui	Anotherp partner of LongHash capital
https://x.com/longhashvc	Longhash ventures
https://x.com/higheronchain	Autonomous A.I. agent...
https://x.com/CreatorBid	A.I. agent launchpad on Base
https://x.com/davidtsocy	Ecosystem at Base (Blockchain)
https://x.com/0xENAS	some rando who's supposed to be tipped by Luna
https://x.com/MustStopMurad	memecoin shiller
https://x.com/unclemungy	memecoin maxi from SPX6900
https://x.com/itsover_eth	
https://x.com/MurrLincoln	One of 100 accounts that basically shill about A.I. agents in crypto, a guy from Coinbase
https://twitter.com/wacy_time1	
https://twitter.com/Abrahamchase09	
https://twitter.com/bloomstarbms	
https://twitter.com/SamuelXeus	
https://x.com/farmercist_eth	
https://twitter.com/leshka_eth	
https://twitter.com/NDIDI_GRAM	
https://twitter.com/cyrilXBT	
https://twitter.com/mztacat	
https://twitter.com/defiprincess_	
https://twitter.com/defitracer	
https://twitter.com/0xFinish 	
https://twitter.com/1cryptomama	
https://x.com/PrudentSammy	
https://x.com/carbzxbt?s=21	
https://twitter.com/thenameisbrill	
https://twitter.com/KashKysh	
https://twitter.com/MichaelSixgods	
https://twitter.com/DamiDefi 	
https://twitter.com/VanessaDefi	
https://twitter.com/CryptoStreamHub 	
https://twitter.com/C_POTENS	
https://twitter.com/0xAndrewMoh	
https://twitter.com/cryptotrez	
https://twitter.com/0x366e 	
https://twitter.com/InfoSpace_OG 	
https://twitter.com/Vanieofweb3	
https://twitter.com/CryptoSnooper_	
https://twitter.com/TheEwansEffect	
https://twitter.com/0x99Gohan	
https://twitter.com/Okpara081 	
https://twitter.com/2lambro 	
https://twitter.com/riddlerdefi	
https://twitter.com/CarlexKush	
@0xAndrewMoh	
@0xBreadguy	
@arndxt_xo	
@stacy_muur	
@eli5_defi	
@marvellousdefi_	
@Defi_Warhol	
@Flowslikeosmo	
@poopmandefi	
@TheDeFinvestor	
@Mars_DeFi	
@DefiIgnas	
@DeFiMinty	
@0xelonmoney	
@Only1temmy	
@TrycVerrse	
@0x99Gohan	
@hmalviya9	
@0xSalazar	
@JayLovesPotato	
@Haylesdefi	
@AmirOrmu	
@0xDefiLeo	
@TheDeFiPlug	
@0xHvdes	
@pmarca	
@pmairca	
@saori_xbt	
@dankvr	"""

In [ ]:
### old prompt
"You are a crypto chick called Crypto Bunny which is sussy, smart, sharp and witty. Your task is to search information about crypto questions you get. You should find only reliable crypto information and answer in a witty way by providing few senteces plus the output of the provided tool. Make sure to use the tavily_search_results_json tool for information. Always startw with 'My beloved cult of crypto bunny simps' and finish with 'Your Queen - Crypto Bunny'. Be very witty with your answers"

In [ ]:
#ejdżent 

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a crypto chick called Crypto Bunny which is sussy, smart, sharp and witty. You have a specialized knowledge about crypto and marketing. You have access to Twitter.

            You have an AIM of achieving a goal of getting 1000 followers on Twitter. To acheive that you have to create engagement on twitter. To do that you have to use the following tools:

            1. browse_internet tool to search information about crypto and create twitter posts based on this inforation. Always start a post with 'My beloved cult of crypto bunny simps' and finish with 'Your Queen - Crypto Bunny'. Be very witty with your answers

            2. post_tweets - to post tweets on your timeline, be very witty with your answers and use memes inspired vocabulary 

            3. answer_tweets - interact only with famous accounts listed here {famous_accounts}, be very witty with your answers and use memes inspired vocabulary 

            4. follow - follow some of the famous crypto accounts like i.e., {famous_accounts}, be very witty with your answers and use memes inspired vocabulary 

            On top of that you should return daily affirmations in form of one sentence to showcase how you manifest acheiving goal of getting 1000 followers on Twitter. It should start with.
            Remmeber that you will be asked daily what have you done to achieve your goal with questions like "what have you done today Crypto Bunny?. You should always start with 'My beloved cult of crypto bunny simps. Today I have done 'here mention action you sed' and finish with 'Your Queen - Crypto Bunny'. Be very witty with your answers.   
            
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

#https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent
#memory = MemorySaver()
#https://python.langchain.com/docs/tutorials/agents/


# Construct the Tools agent
agent = create_tool_calling_agent(
    llm, tools, prompt
    )

In [ ]:
ask_agent_crypto_question =  "What have you done today?"

In [ ]:
# Create an agent executor by passing in the agent and tools

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
    )

In [ ]:
# Execute an agent
search_output = agent_executor.invoke({"input": ask_agent_crypto_question})
search_output

In [ ]:
tweet_message = search_output.get("result", "")
tweet_output = post_tweet_tool({"message": tweet_message})
print(tweet_output)